In [10]:
!python -m pip install pypdfium2
!python -m pip install tqdm


[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: C:\Users\nickk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.6 kB 222.6 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/57.6 kB 220.2 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 303.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.4 kB ? eta -:--:--
   ------------------------------------ --- 71.7/78.4 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 78.4/78.4 kB 1.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: C:\Users\nickk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pypdfium2 as pdfium
import pypdfium2.raw as pdfium_c
from tqdm import tqdm  # New import for progress tracking

# --- 1. Styling Definitions ---
class Style:
    """Holds configuration for UI elements."""
    GRAY_LINE = {
        'color': (180, 180, 180, 255), # RGBA
        'width': 1.0,
    }
    NOTE_ZONE = {
        'color': (250, 250, 250, 255), # Very light gray
        'fill': True
    }

# --- 2. Drawing Library (Logic remains same) ---
class PDFDrawer:
    @staticmethod
    def add_vertical_line(page, x, y_start, y_end, style=Style.GRAY_LINE):
        path = pdfium_c.FPDFPageObj_CreateNewPath(x, y_start)
        pdfium_c.FPDFPath_LineTo(path, x, y_end)
        r, g, b, a = style['color']
        pdfium_c.FPDFPageObj_SetStrokeColor(path, r, g, b, a)
        pdfium_c.FPDFPageObj_SetStrokeWidth(path, style['width'])
        pdfium_c.FPDFPath_SetDrawMode(path, pdfium_c.FPDF_FILLMODE_NONE, 1)
        pdfium_c.FPDFPage_InsertObject(page.raw, path)

    @staticmethod
    def add_rect(page, left, bottom, right, top, style=Style.NOTE_ZONE):
        rect = pdfium_c.FPDFPageObj_CreateNewPath(left, bottom)
        pdfium_c.FPDFPath_LineTo(rect, left, top)
        pdfium_c.FPDFPath_LineTo(rect, right, top)
        pdfium_c.FPDFPath_LineTo(rect, right, bottom)
        pdfium_c.FPDFPath_Close(rect)
        r, g, b, a = style['color']
        pdfium_c.FPDFPageObj_SetFillColor(rect, r, g, b, a)
        pdfium_c.FPDFPath_SetDrawMode(rect, pdfium_c.FPDF_FILLMODE_ALTERNATE, 0)
        pdfium_c.FPDFPage_InsertObject(page.raw, rect)

# --- 3. The Orchestrator with Progress Bar ---
class TextbookEnhancer:
    def __init__(self, input_path):
        self.pdf = pdfium.PdfDocument(input_path)
        self.drawer = PDFDrawer()

    def apply_margin_enhancement(self, extra_width=160, margin_boundary=False, shade_margin=False):
        print(f"Enhancing {len(self.pdf)} pages...")
        
        # Wrap the iterator in tqdm for the progress bar
        for page in tqdm(self.pdf, desc="Processing Pages", unit="page"):
            left, bottom, right, top = page.get_mediabox()
            new_right = right + extra_width
            
            page.set_mediabox(left, bottom, new_right, top)
            page.set_cropbox(left, bottom, new_right, top)

            if shade_margin:
                self.drawer.add_rect(page, right, bottom, new_right, top)

            if margin_boundary:
                self.drawer.add_vertical_line(page, right, bottom, top)
            
            pdfium_c.FPDFPage_GenerateContent(page.raw)

    def save(self, output_path):
        print(f"Saving to {output_path}...")
        self.pdf.save(output_path)

if __name__ == "__main__":
    # Example execution
    try:
        enhancer = TextbookEnhancer("math_book.pdf")
        enhancer.apply_margin_enhancement(extra_width=200)
        enhancer.save("enhanced_textbook.pdf")
        print("Success.")
    except Exception as e:
        print(f"Error: {e}")

Enhancing 852 pages...


Processing Pages: 100%|██████████| 852/852 [00:01<00:00, 782.57page/s]


Saving to enhanced_textbook.pdf...
Success.
